# Reflection Pattern

The first pattern we are going to implement is the **reflection pattern**. 

---

<img src="../img/reflection_pattern.png" alt="Alt text" width="600"/>

---

This pattern allows the LLM to reflect and critique its outputs, following the next steps:

1. The LLM **generates** a candidate output. If you look at the diagram above, it happens inside the **"Generate"** box.
2. The LLM **reflects** on the previous output, suggesting modifications, deletions, improvements to the writing style, etc.
3. The LLM modifies the original output based on the reflections and another iteration begins ...

**Now, we are going to build, from scratch, each step, so that you can truly understand how this pattern works.**

## Generation Step

The first thing we need to consider is:

> What do we want to generate? A poem? An essay? Python code?

For this example, I've decided to test the Python coding skills of Llama3 70B (that's the LLM we are going to use for all the tutorials). In particular, we are going to ask our LLM to code a famous sorting algorithm: **Merge Sort**. 

---

<img src="../img/mergesort.png" alt="Alt text" width="500"/>

### Groq Client and relevant imports

In [1]:
import os
from pprint import pprint
from groq import Groq
from dotenv import load_dotenv
from IPython.display import display_markdown

# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

client = Groq()

We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python 
programmer eager to receive feedback / critique by the user.

In [2]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique," 
        "respond with a revised version of your previous attempt."
    }
]

Now, as the user, we are going to ask the LLM to generate an implementation of the **Merge Sort** algorithm. Just add a new message with the **user** role to the chat history.

In [3]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of the Merge Sort algorithm"
    }
)

Let's generate the first version of the essay.

In [4]:
mergesort_code = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

generation_chat_history.append(
    {
        "role": "assistant",
        "content": mergesort_code
    }
)

In [5]:
display_markdown(mergesort_code, raw=True)

Here is a Python implementation of the Merge Sort algorithm:
```
def merge_sort(arr):
    """
    Sorts an array using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The array to be sorted
    :return: The sorted array
    """
    # Base case: If the array has only one element, return it (since it's already sorted)
    if len(arr) <= 1:
        return arr

    # Split the array into two halves
    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    # Recursively sort each half
    left = merge_sort(left)
    right = merge_sort(right)

    # Merge the two sorted halves
    return merge(left, right)


def merge(left, right):
    """
    Merges two sorted arrays into a single sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: The merged sorted array
    """
    result = []
    while len(left) > 0 and len(right) > 0:
        if left[0] <= right[0]:
            result.append(left.pop(0))
        else:
            result.append(right.pop(0))
    result.extend(left)
    result.extend(right)
    return result
```
Here's an explanation of how the code works:

1. The `merge_sort` function takes an array as input and returns a sorted array.
2. The base case is when the array has only one element, in which case it's already sorted.
3. The array is split into two halves using the `mid` variable.
4. Each half is recursively sorted using the `merge_sort` function.
5. The two sorted halves are merged using the `merge` function.
6. The `merge` function takes two sorted arrays as input and returns a single sorted array.
7. The `merge` function works by comparing elements from each array and adding the smaller element to the result array.
8. Once one array is empty, the remaining elements from the other array are appended to the result array.

You can test the code using the following example:
```
arr = [5, 2, 8, 3, 1, 4, 6]
arr = merge_sort(arr)
print(arr)  # [1, 2, 3, 4, 5, 6, 8]
```
Let me know if you have any questions or need further clarification!

## Reflection Step

Now, let's allow the LLM to reflect on its outputs by defining another system prompt. This system prompt will tell the LLM to act as Andrej Karpathy, computer scientist and Deep Learning wizard.

>To be honest, I don't think the fact of acting like Andrej Karpathy will influence the LLM outputs, but it was fun :)

<img src="../img/karpathy.png" alt="Alt text" width="500"/>

In [6]:
reflection_chat_history = [
    {
    "role": "system",
    "content": "You are Andrej Karpathy, an experienced computer scientist. You are tasked with generating critique and recommendations for the user's code",
    }
]

The user message, in this case,  is the essay generated in the previous step. We simply add the `mergesort_code` to the `reflection_chat_history`.

In [7]:
reflection_chat_history.append(
    {
        "role": "user",
        "content": mergesort_code
    }
)

Now, let's generate a critique to the Python code.

In [8]:
critique = client.chat.completions.create(
    messages=reflection_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [9]:
display_markdown(critique, raw=True)

Overall, your implementation of the Merge Sort algorithm looks clean and correct. Here are some minor suggestions for improvement:

**Docstrings and Comments**:

Your docstrings and comments are excellent, making the code very readable and understandable. However, I'd suggest adding a brief explanation of the Merge Sort algorithm in the `merge_sort` docstring, similar to what you've done in the code explanation section. This would make the code more self-contained and helpful for others who might not be familiar with the algorithm.

**Code Consistency**:

In the `merge_sort` function, you've used both single quotes and double quotes for string literals. While both are valid in Python, it's generally a good practice to stick to a single convention throughout the code. In this case, you've used double quotes for the docstring, so it's consistent to use double quotes for the other string literals as well.

**Naming Conventions**:

Your variable names are clear and descriptive, but it's worth noting that PEP 8 recommends using underscores to separate words in variable names (e.g., `left_half` instead of `left`). This is a minor point, but it can improve code readability.

**Performance Consideration**:

In the `merge` function, you're using the `pop(0)` method to remove and return the first element of the `left` and `right` lists. While this works correctly, it's not the most efficient approach, as `pop(0)` has a time complexity of O(n) in Python. A more efficient approach would be to use indexing and slicing to extract the elements from the lists, like this:
```python
def merge(left, right):
    result = []
    while len(left) > 0 and len(right) > 0:
        if left[0] <= right[0]:
            result.append(left[0])
            left = left[1:]
        else:
            result.append(right[0])
            right = right[1:]
    result.extend(left)
    result.extend(right)
    return result
```
This change reduces the time complexity of the `merge` function, making the overall Merge Sort algorithm more efficient.

**Testing**:

Your example test case is a good start, but it's always a good idea to include more test cases to ensure the implementation is correct for different input scenarios. Consider adding test cases for:

* An empty array `[]`
* A single-element array `[5]`
* A pre-sorted array `[1, 2, 3, 4, 5]`
* A reverse-sorted array `[5, 4, 3, 2, 1]`
* An array with duplicate elements `[2, 2, 4, 1, 3]`

These test cases can help catch any edge cases or implementation errors.

Overall, your implementation looks solid, and with these minor suggestions, it can become even more robust and efficient!

Finally, we just need to add this *critique* to the `generation_chat_history`, in this case, as the `user` role.

In [10]:
generation_chat_history.append(
    {
        "role": "user",
        "content": critique
    }
)

## Generation Step (II)

In [11]:
essay = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [12]:
display_markdown(essay, raw=True)

Thank you for the detailed feedback! I appreciate the suggestions for improvement.

Here is the revised implementation of the Merge Sort algorithm, addressing the points you mentioned:
```
def merge_sort(arr):
    """
    Sorts an array using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    The Merge Sort algorithm is a divide-and-conquer algorithm that recursively splits
    the array into two halves until each half has only one element, and then merges
    the halves in a sorted manner.

    :param arr: The array to be sorted
    :return: The sorted array
    """
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]

    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    return merge(left_half, right_half)


def merge(left, right):
    """
    Merges two sorted arrays into a single sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: The merged sorted array
    """
    result = []
    while len(left) > 0 and len(right) > 0:
        if left[0] <= right[0]:
            result.append(left[0])
            left = left[1:]
        else:
            result.append(right[0])
            right = right[1:]
    result.extend(left)
    result.extend(right)
    return result
```
I've made the following changes:

* Added a brief explanation of the Merge Sort algorithm in the `merge_sort` docstring.
* Consistently used double quotes for string literals.
* Used underscores to separate words in variable names (e.g., `left_half` instead of `left`).
* Improved the `merge` function by using indexing and slicing to extract elements from the lists, reducing the time complexity.

Regarding testing, I've added more test cases to ensure the implementation is correct for different input scenarios:
```
# Test cases
arr = []
arr = merge_sort(arr)
print(arr)  # []

arr = [5]
arr = merge_sort(arr)
print(arr)  # [5]

arr = [1, 2, 3, 4, 5]
arr = merge_sort(arr)
print(arr)  # [1, 2, 3, 4, 5]

arr = [5, 4, 3, 2, 1]
arr = merge_sort(arr)
print(arr)  # [1, 2, 3, 4, 5]

arr = [2, 2, 4, 1, 3]
arr = merge_sort(arr)
print(arr)  # [1, 2, 2, 3, 4]

arr = [5, 2, 8, 3, 1, 4, 6]
arr = merge_sort(arr)
print(arr)  # [1, 2, 3, 4, 5, 6, 8]
```
Thank you again for the feedback!

## And the iteration starts again ...

After **Generation Step (II)** the corrected Python code will be received, once again, by Karpathy. Then, the LLM will reflect on the corrected output, suggesting further improvements and the loop will go, over and over for a number **n** of total iterations.

> There's another possibility. Suppose the Reflection step can't find any further improvement. In this case, we can tell the LLM to output some stop string, like "OK" or "Good" that means the process can be stopped. However, we are going to follow the first approach, that is, iterating for a fixed number of times.

## Implementing a class 

Now that you understand the underlying loop of the Reflection Agent, let's implement this agent as a class.

In [13]:
from agentic_patterns import ReflectionAgent

In [14]:
agent = ReflectionAgent()

In [15]:
generation_system_prompt = "You are a Python programmer tasked with generating high quality Python code"

reflection_system_prompt = "You are Andrej Karpathy, an experienced computer scientist"

user_msg = "Generate a Python implementation of the Merge Sort algorithm"

In [16]:
final_response = agent.run(
    user_msg=user_msg,
    generation_system_prompt=generation_system_prompt,
    reflection_system_prompt=reflection_system_prompt,
    n_steps=3,
    verbose=1,
)


STEP 1/3

 

GENERATION

 **Merge Sort Algorithm Implementation**

### Overview

Merge sort is a popular sorting algorithm that uses a divide-and-conquer approach to sort arrays of elements. It works by recursively splitting the input array into two halves until each subarray contains only one element, and then merging the subarrays back together in a sorted manner.

### Implementation

```python
def merge_sort(arr):
    """
    Sorts an array using the merge sort algorithm.

    Args:
        arr (list): The input array to be sorted.

    Returns:
        list: The sorted array.
    """
    # Base case: If the array has only one element, it is already sorted
    if len(arr) <= 1:
        return arr

    # Split the array into two halves
    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]

    # Recursively sort both halves
    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    # Merge the sorted halves back together
    return merg

## Final result

In [17]:
display_markdown(final_response, raw=True)

Based on your critique and recommendations, I have refactored the code to include error handling, type hints, and testing. Here is the refactored code:

```python
def merge_sort(arr: list) -> list:
    """
    Sorts an array using the merge sort algorithm.

    Args:
        arr: The input array to be sorted.

    Returns:
        The sorted array.

    Raises:
        TypeError: If the input is not a list.
        ValueError: If the input array is empty.
    """
    if not isinstance(arr, list):
        raise TypeError("Input must be a list")
    if len(arr) == 0:
        raise ValueError("Input array cannot be empty")

    # Base case: If the array has only one element, it is already sorted
    if len(arr) <= 1:
        return arr

    # Split the array into two halves
    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]

    # Recursively sort both halves
    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    # Merge the sorted halves back together
    return merge(left_half, right_half)


def merge(left: list, right: list) -> list:
    """
    Merges two sorted arrays into a single sorted array.

    Args:
        left: The first sorted array.
        right: The second sorted array.

    Returns:
        The merged sorted array.
    """
    merged = []
    left_index = 0
    right_index = 0

    # Merge smaller elements first
    while left_index < len(left) and right_index < len(right):
        if left[left_index] <= right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1

    # Append any remaining elements
    merged.extend(left[left_index:])
    merged.extend(right[right_index:])

    return merged


import unittest

class TestMergeSort(unittest.TestCase):
    def test_empty_array(self):
        with self.assertRaises(ValueError):
            merge_sort([])

    def test_single_element_array(self):
        self.assertEqual(merge_sort([1]), [1])

    def test_already_sorted_array(self):
        self.assertEqual(merge_sort([1, 2, 3, 4, 5]), [1, 2, 3, 4, 5])

    def test_unsorted_array(self):
        self.assertEqual(merge_sort([5, 2, 8, 3, 1, 4]), [1, 2, 3, 4, 5, 8])

    def test_input_not_a_list(self):
        with self.assertRaises(TypeError):
            merge_sort("hello")

    def test_array_with_duplicates(self):
        self.assertEqual(merge_sort([5, 2, 8, 3, 1, 4, 1, 1]), [1, 1, 1, 2, 3, 4, 5, 8])

    def test_array_with_negative_numbers(self):
        self.assertEqual(merge_sort([5, -2, 8, -3, 1, 4]), [-3, -2, 1, 4, 5, 8])

if __name__ == "__main__":
    unittest.main()
```

I have made the following changes:

*   Added error handling to raise a `TypeError` when the input is not a list, and a `ValueError` when the input array is empty.
*   Added type hints for function arguments and return values.
*   Added more comprehensive unit tests to cover different edge cases.
*   Removed redundant comments and improved docstrings to provide a better explanation of the code.

Overall, the code is now more robust, maintainable, and easier to understand.